In [21]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [22]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2[df2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [23]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]

In [24]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [25]:
hankel = Hankel()
lag = 60
stride = 0.5

In [26]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [27]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
Y_test = test_combined.loc[:,'ATT_FLAG']

In [28]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [29]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for sens in sensors:
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [30]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [31]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.788235294117647
Precision  0.7692307692307693
Recall  0.8695652173913043
F1-score  0.8163265306122449


In [32]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5034199726402189 Median : 0.5058823529411764 Min : 0.4470588235294118 Max : 0.6235294117647059
precision - Mean: 0.6852574750830563 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.09049544994944388 Median : 0.08695652173913043 Min : 0.0 Max : 0.32608695652173914
f1 -        Mean: 0.15515775874080093 Median : 0.16 Min : 0.0 Max : 0.48387096774193544


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [33]:
fscores_k = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)  
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int) 
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.7529411764705882
Precision  0.7551020408163265
Recall  0.8043478260869565
F1-score  0.7789473684210526
-------number of clusters------ 3
Accuracy  0.7529411764705882
Precision  0.7450980392156863
Recall  0.8260869565217391
F1-score  0.7835051546391751
-------number of clusters------ 4
Accuracy  0.7411764705882353
Precision  0.7222222222222222
Recall  0.8478260869565217
F1-score  0.78
-------number of clusters------ 5
Accuracy  0.7647058823529411
Precision  0.7321428571428571
Recall  0.8913043478260869
F1-score  0.8039215686274508
-------number of clusters------ 6
Accuracy  0.7058823529411765
Precision  0.6721311475409836
Recall  0.8913043478260869
F1-score  0.7663551401869158
-------number of clusters------ 7
Accuracy  0.6588235294117647
Precision  0.6307692307692307
Recall  0.8913043478260869
F1-score  0.7387387387387386


In [34]:
fscores_k

[0.7789473684210526,
 0.7835051546391751,
 0.78,
 0.8039215686274508,
 0.7663551401869158,
 0.7387387387387386]

# Multiple clusters + No threshold tuning (No concat of training and test data)

In [35]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']

In [36]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [37]:
fscores_k_2=[]
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_2.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.6764705882352942
Precision  0.5945945945945946
Recall  0.7586206896551724
F1-score  0.6666666666666667
-------number of clusters------ 3
Accuracy  0.6029411764705882
Precision  0.525
Recall  0.7241379310344828
F1-score  0.6086956521739131
-------number of clusters------ 4
Accuracy  0.5588235294117647
Precision  0.4888888888888889
Recall  0.7586206896551724
F1-score  0.5945945945945945
-------number of clusters------ 5
Accuracy  0.6176470588235294
Precision  0.5319148936170213
Recall  0.8620689655172413
F1-score  0.6578947368421053
-------number of clusters------ 6
Accuracy  0.6323529411764706
Precision  0.5454545454545454
Recall  0.8275862068965517
F1-score  0.6575342465753425
-------number of clusters------ 7
Accuracy  0.6470588235294118
Precision  0.5471698113207547
Recall  1.0
F1-score  0.7073170731707317


In [38]:
fscores_k_2

[0.6666666666666667,
 0.6086956521739131,
 0.5945945945945945,
 0.6578947368421053,
 0.6575342465753425,
 0.7073170731707317]

# Multiple clusters + Threshold tuning (No concat of training and test data)

In [39]:
fscores_k_3=[]
for k in range(2,10):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,kscore_init='inertia')
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_3.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 3
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 4
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 5
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 6
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 7
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 8
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 9
Accuracy  0.4264705882352941

In [40]:
fscores_k_3

[0.5979381443298969,
 0.5979381443298969,
 0.5979381443298969,
 0.5979381443298969,
 0.5979381443298969,
 0.5979381443298969,
 0.5979381443298969,
 0.5979381443298969]